In [1]:
#scraping the VU course catalog

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By

base_url = 'https://studiegids.vu.nl/en/courses/'



In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")
options.add_argument("--incognito")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")



for year in [2021, 2022, 2023]:

    course_ids = []

    print('year: ', year)
    url = base_url + str(year) + '-' + str(year+1) + '/'
    print(url)

    #use selenium chrome to get the page source
    driver = webdriver.Chrome( options=options)
    driver.get(url)

    #wait 1 sec
    time.sleep(1)
    #scroll all the way down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #find the pages total
    nav = driver.find_elements(By.CLASS_NAME, 'sg-pagination')[1]
    ul = nav.find_element(By.TAG_NAME, 'ul')
    lis = ul.find_elements(By.TAG_NAME, 'li')
    pages = int(lis[-2].text)

    print('pages: ', pages)

    for page in tqdm(range(pages)):
        #get all li sg-search-result
        attempts = 0
        while attempts < 3:
            try:
                results = driver.find_elements(By.XPATH, '//li[@class="sg-search-result"]')
                break
            except:
                attempts += 1
                print('retrying')
                time.sleep(1)

        #get the url for each course from the a inside the result
        for result in results:
            a = result.find_element(By.TAG_NAME, 'a')
            course_ids.append(a.get_attribute('href'))
            
        #find the next arrow again
        nav = driver.find_elements(By.CLASS_NAME, 'sg-pagination')[1]
        ul = nav.find_element(By.TAG_NAME, 'ul')
        lis = ul.find_elements(By.TAG_NAME, 'li')
        next = lis[-1]

        #click the last li
        next.click()


        #wait 3 seconds
        time.sleep(3)

    #close the driver
    driver.close()

    #save the course ids to pickle of the year in the data folder
    pd.to_pickle(course_ids, 'data/course_ids_' + str(year) + '.pkl')
    print('saved course ids for year: ', year)
    print('number of courses: ', len(course_ids))
    print()
    print()






year:  2021
https://studiegids.vu.nl/en/courses/2021-2022/
pages:  244


  0%|          | 0/244 [00:00<?, ?it/s]

saved course ids for year:  2021
number of courses:  3688


year:  2022
https://studiegids.vu.nl/en/courses/2022-2023/
pages:  246


  0%|          | 0/246 [00:00<?, ?it/s]

saved course ids for year:  2022
number of courses:  3725


year:  2023
https://studiegids.vu.nl/en/courses/2023-2024/
pages:  249


  0%|          | 0/249 [00:00<?, ?it/s]

saved course ids for year:  2023
number of courses:  3756




FileNotFoundError: [Errno 2] No such file or directory: '/data/course_ids_2019.pkl'

In [5]:
#merge all the pickles
course_ids = []
for year in [2019, 2020, 2021, 2022, 2023]:
    course_ids += pd.read_pickle('data/course_ids_' + str(year) + '.pkl')

#save the merged pickle
pd.to_pickle(course_ids, 'data/course_ids.pkl')